In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define the folder name and change directory
FOLDERNAME = 'Colab Notebooks/Rec_Sys'
%cd /content/drive/MyDrive/{FOLDERNAME}

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Rec_Sys


In [ ]:
!pip install imbalanced-learn

In [ ]:
!pip install fastFM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastFM: filename=fastFM-0.2.10-cp310-cp310-linux_x86_64.whl size=591481 sha256=3bef159fd14bbddec9601830ed6c2b48f68e92e0d3f530ee6bc8feb00e291213
  Stored in directory: /root/.cache/pip/wheels/93/92/52/2da7997fcb7a7ce9042ff3b33836ef0c2fd47aa95382d7a113
Successfully built fastFM


In [ ]:
import gzip
from collections import defaultdict
from sklearn import svm
import numpy
import random
import string
from sklearn import linear_model
import pandas as pd
from numpy.random import normal
from sklearn.model_selection import GridSearchCV
import scipy
import tensorflow as tf
from fastFM import als
from scipy.spatial import distance
import json

In [ ]:
FILEPATH = 'renttherunway_final_data.json.gz'
META = 'endomondoMeta.json.gz'

In [ ]:
def readJSON(path):
    with gzip.open(path, 'rt', encoding='utf-8') as f:
        f.readline()
        for line in f:
            d = json.loads(line)
            yield d

In [ ]:
raw = []
for l in readJSON(FILEPATH):
    raw.append(l)

In [ ]:
raw_dict = [item for item in raw]

df = pd.DataFrame(raw_dict)

In [ ]:
df = df.drop(["review_text", "review_summary", "review_date"], axis = 1)

In [ ]:
na_value = df.isna()
na_sum = na_value.sum(axis = 0)
na_sum

fit               0
user_id           0
bust size     18411
item_id           0
weight        29982
rating           82
rented for       10
body type     14637
category          0
height          677
size              0
age             960
dtype: int64

In [ ]:
# for categorical data, use mode to replace the null value
bustsize_mode = df['bust size'].mode()[0]
bodytype_mode = df['body type'].mode()[0]
df['bust size'].fillna(bustsize_mode, inplace=True)
df['body type'].fillna(bodytype_mode, inplace=True)

In [ ]:
# make sure weight column has same unit
df["weight"].str.extract('\d+(...)').value_counts()

lbs    162561
dtype: int64

In [ ]:
# for numerical data, use global mean to replace the null value
df["weight"] = df['weight'].str.extract('(\d+)').astype(float)
weight_mean = df["weight"].mean()
df["weight"].fillna(weight_mean, inplace=True)

df["age"] = df["age"].astype(float)
age_mean = df["age"].mean()
df["age"].fillna(age_mean, inplace=True)

In [ ]:
def feet_inches_to_cm(s):
    # Check if 's' is a string
    if isinstance(s, str):
        parts = s.split("'")
        feet = int(parts[0])
        inches = int(parts[1].replace('"', '').strip())
        return (feet * 30.48) + (inches * 2.54)
    else: # if 's' is not a string
        return None

In [ ]:
df['height_cm'] = df['height'].apply(feet_inches_to_cm)

df["height_cm"] = df["height_cm"].astype(float)
height_mean = df["height_cm"].mean()
df["height_cm"].fillna(height_mean, inplace=True)

In [ ]:
df = df.drop("height", axis = 1)

In [ ]:
na_value = df.isna()
na_sum = na_value.sum(axis = 0)
na_sum

fit            0
user_id        0
bust size      0
item_id        0
weight         0
rating        82
rented for    10
body type      0
category       0
size           0
age            0
height_cm      0
dtype: int64

In [ ]:
data = df.dropna()

In [ ]:
data["fit"] = data["fit"].astype("category")
data["bust size"] = data["bust size"].astype("category")
data["rating"] = data["rating"].astype(float)
data["rented for"] = data["rented for"].astype("category")
data["body type"] = data["body type"].astype("category")
data["category"] = data["category"].astype("category")

<ipython-input-19-9a9a8a73c235>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["fit"] = data["fit"].astype("category")
<ipython-input-19-9a9a8a73c235>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["bust size"] = data["bust size"].astype("category")
<ipython-input-19-9a9a8a73c235>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [ ]:
data = data.to_dict(orient='records')

In [ ]:
type(data)

list

In [ ]:
len(data)

192451

In [ ]:
data[0]

{'fit': 'fit',
 'user_id': '273551',
 'bust size': '34b',
 'item_id': '153475',
 'weight': 132.0,
 'rating': 10.0,
 'rented for': 'other',
 'body type': 'straight & narrow',
 'category': 'gown',
 'size': 12,
 'age': 36.0,
 'height_cm': 167.64000000000001}

In [ ]:
random.shuffle(data)

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.sparse import hstack, lil_matrix
import numpy as np
from fastFM import als

In [ ]:
# Separating numerical and categorical columns
# 'fit', 'user_id', 'bust size', 'item_id', 'rented for', 'body type', 'category'
numerical_cols = ['weight', 'size', 'age', 'height_cm']
categorical_cols = ['fit', 'user_id', 'item_id', 'rented for', 'body type', 'category']

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,body type,category,size,age,height_cm
0,fit,890577,32b,164051,137.391711,8.0,party,petite,dress,8,33.0,160.02
1,large,867427,34dd,2443818,137.000000,8.0,work,full bust,top,14,27.0,162.56
2,fit,87400,34b,613085,119.000000,10.0,everyday,athletic,dress,4,32.0,170.18
3,fit,325697,34b,1076484,137.391711,10.0,wedding,hourglass,dress,12,33.0,170.18
4,fit,544841,36c,1108814,145.000000,8.0,wedding,hourglass,dress,14,33.0,160.02


In [ ]:
import pandas as pd
from fastFM import als
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.sparse import hstack

In [ ]:
userIDs,itemIDs = {},{}

for d in data:
    u,i = d['user_id'],d['item_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers,nItems = len(userIDs),len(itemIDs)

In [ ]:
nUsers,nItems

(105504, 5850)

predict fit/small/large

# A: fit:1, else:-1

In [ ]:
# One-hot encoding a categorical feature and normalizing a numerical feature
encoder = OneHotEncoder()
X_categorical = encoder.fit_transform(df[['user_id', 'item_id', 'bust size', 'rented for', 'body type', 'category']])
scaler = StandardScaler()
X_numerical = scaler.fit_transform(df[['weight', 'rating', 'size', 'age', 'height_cm']])

In [ ]:
# Combine categorical and numerical features
X_init = hstack([X_categorical, X_numerical])
X = scipy.sparse.lil_matrix(X_init)

# Prepare the target variable
# y_a = df['fit_a'].values

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, df['fit'], test_size=0.25)

In [ ]:
!pip install Optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.0 MB/s eta 0:00:00


In [ ]:
df['fit'].value_counts()

fit      141985
small     25776
large     24690
Name: fit, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE

# Assume X_train and y_train are your features and labels
smote = SMOTE(sampling_strategy={'small': 100000, 'large': 100000})
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
from collections import Counter
count = Counter(y_train_smote)
count

Counter({'fit': 106483, 'small': 100000, 'large': 100000})

In [ ]:
# Map 'fit' to 1 and both 'small' and 'large' to -1
# df['fit_a'] = df['fit'].map({'fit': 1, 'small': -1, 'large': -1})
y_train_smote = y_train_smote.replace({'fit': 1, 'small': -1, 'large': -1})
y_test = y_test.replace({'fit': 1, 'small': -1, 'large': -1})

In [ ]:
import optuna
from fastFM import sgd
from sklearn.metrics import accuracy_score
import numpy as np

def objective(trial):
    # Define the hyperparameters to be tuned
    n_iter = trial.suggest_int('n_iter', 500, 20000)
    init_stdev = trial.suggest_float('init_stdev', 0.0001, 0.25)
    l2_reg_w = trial.suggest_float('l2_reg_w', 0.0, 0.2)
    l2_reg_V = trial.suggest_float('l2_reg_V', 0.0, 20)
    rank = trial.suggest_int('rank', 1, 5)
    step_size = trial.suggest_float('step_size', 0.001, 1)

    # Initialize and train the Factorization Machine model
    fm = sgd.FMClassification(n_iter=n_iter, init_stdev=init_stdev,
                              l2_reg_w=l2_reg_w, l2_reg_V=l2_reg_V,
                              rank=rank, step_size=step_size)
    fm.fit(X_train_smote, y_train_smote)

    # Make predictions and evaluate the model
    y_pred = fm.predict(X_test)
    score = accuracy_score(y_test, np.round(y_pred))  # Assuming binary classification

    return score

# Run Optuna optimization
study = optuna.create_study(direction='maximize')  # Use 'minimize' for metrics like RMSE
study.optimize(objective, n_trials=1000)

# Best hyperparameters
best_params = study.best_params
print('Best parameters:', best_params)


In [ ]:
best_params = {'n_iter': 16375, 'init_stdev': 0.008375211915842371, 'l2_reg_w': 0.03261634530835802, 'l2_reg_V': 4.537985558701806, 'rank': 2, 'step_size': 0.01918408357359382}

In [ ]:
print(best_params)

{'n_iter': 16375, 'init_stdev': 0.008375211915842371, 'l2_reg_w': 0.03261634530835802, 'l2_reg_V': 4.537985558701806, 'rank': 2, 'step_size': 0.01918408357359382}


In [ ]:
from fastFM import sgd
# Initialize and train the Factorization Machine model
fm = sgd.FMClassification(n_iter=best_params['n_iter'], init_stdev=best_params['init_stdev'], l2_reg_w=best_params['l2_reg_w'],
                          l2_reg_V=best_params['l2_reg_V'], rank=best_params['rank'], step_size=best_params['step_size'])
fm.fit(X_train_smote, y_train_smote)

# Make predictions
y_pred_a = fm.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_a)
print("Accuracy:", accuracy)

Accuracy: 0.7476565585184878


In [ ]:
pd.Series(y_pred_a).value_counts()

 1.0    45577
-1.0     2536
dtype: int64

In [ ]:
pd.Series(y_test).value_counts()

 1    35502
-1    12611
Name: fit, dtype: int64

# B: small:1, else:-1

In [ ]:
# # Map 'small' to 1 and both 'fit' and 'large' to -1
# df['fit_b'] = df['fit'].map({'fit': -1, 'small': 1, 'large': -1})

In [ ]:
# One-hot encoding a categorical feature and normalizing a numerical feature
encoder = OneHotEncoder()
X_categorical = encoder.fit_transform(df[['user_id', 'item_id', 'bust size', 'rented for', 'body type', 'category']])
scaler = StandardScaler()
X_numerical = scaler.fit_transform(df[['weight', 'rating', 'size', 'age', 'height_cm']])

In [ ]:
# Combine categorical and numerical features
X_init = hstack([X_categorical, X_numerical])
X = scipy.sparse.lil_matrix(X_init)

# Prepare the target variable
# y_b = df['fit_b'].values

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, df['fit'], test_size=0.25)

In [ ]:
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Map 'fit' to 1 and both 'small' and 'large' to -1
y_train_smote = y_train_smote.replace({'fit': -1, 'small': 1, 'large': -1})
y_test = y_test.replace({'fit': -1, 'small': 1, 'large': -1})

In [ ]:
import optuna
from fastFM import sgd
from sklearn.metrics import accuracy_score
import numpy as np

def objective(trial):
    # Define the hyperparameters to be tuned
    n_iter = trial.suggest_int('n_iter', 500, 20000)
    init_stdev = trial.suggest_float('init_stdev', 0.0001, 0.25)
    l2_reg_w = trial.suggest_float('l2_reg_w', 0.0, 0.2)
    l2_reg_V = trial.suggest_float('l2_reg_V', 0.0, 20)
    rank = trial.suggest_int('rank', 1, 5)
    step_size = trial.suggest_float('step_size', 0.001, 1)

    # Initialize and train the Factorization Machine model
    fm = sgd.FMClassification(n_iter=n_iter, init_stdev=init_stdev,
                              l2_reg_w=l2_reg_w, l2_reg_V=l2_reg_V,
                              rank=rank, step_size=step_size)
    fm.fit(X_train_smote, y_train_smote)

    # Make predictions and evaluate the model
    y_pred = fm.predict(X_test)
    score = accuracy_score(y_test, np.round(y_pred))  # Assuming binary classification

    return score

# Run Optuna optimization
study = optuna.create_study(direction='maximize')  # Use 'minimize' for metrics like RMSE
study.optimize(objective, n_trials=1000)

# Best hyperparameters
best_params_2 = study.best_params
print('Best parameters:', best_params_2)


[I 2023-12-01 01:31:19,261] A new study created in memory with name: no-name-a20ba5b9-2407-4e85-8df0-309b53e0e244
[I 2023-12-01 01:31:19,492] Trial 0 finished with value: 0.8655456945108391 and parameters: {'n_iter': 12691, 'init_stdev': 0.10288414126289461, 'l2_reg_w': 0.1297308923308542, 'l2_reg_V': 4.646995247639623, 'rank': 4, 'step_size': 0.13874876392242444}. Best is trial 0 with value: 0.8655456945108391.
[I 2023-12-01 01:31:19,647] Trial 1 finished with value: 0.8662107954191175 and parameters: {'n_iter': 6996, 'init_stdev': 0.005391821776703288, 'l2_reg_w': 0.18651262053130852, 'l2_reg_V': 15.903971398075242, 'rank': 1, 'step_size': 0.6514105557584084}. Best is trial 1 with value: 0.8662107954191175.
[I 2023-12-01 01:31:19,851] Trial 2 finished with value: 0.8662107954191175 and parameters: {'n_iter': 8565, 'init_stdev': 0.03639019249401952, 'l2_reg_w': 0.09285667693264664, 'l2_reg_V': 15.773280658467954, 'rank': 3, 'step_size': 0.21010224972521102}. Best is trial 1 with value

Best parameters: {'n_iter': 17760, 'init_stdev': 0.015356574969710267, 'l2_reg_w': 0.061908056125114796, 'l2_reg_V': 10.359663485454142, 'rank': 2, 'step_size': 0.18014422523806783}


In [ ]:
print(best_params_2)

{'n_iter': 17760, 'init_stdev': 0.015356574969710267, 'l2_reg_w': 0.061908056125114796, 'l2_reg_V': 10.359663485454142, 'rank': 2, 'step_size': 0.18014422523806783}


In [ ]:
from fastFM import sgd
# Initialize and train the Factorization Machine model
fm = sgd.FMClassification(n_iter=best_params_2['n_iter'], init_stdev=best_params_2['init_stdev'], l2_reg_w=best_params_2['l2_reg_w'],
                          l2_reg_V=best_params_2['l2_reg_V'], rank=best_params_2['rank'], step_size=best_params_2['step_size'])
fm.fit(X_train_smote, y_train_smote)

# Make predictions
y_pred_b = fm.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_b)
print("Accuracy:", accuracy)

Accuracy: 0.8671253091680003


In [ ]:
pd.Series(y_pred_b).value_counts()

-1.0    47965
 1.0      148
dtype: int64

In [ ]:
pd.Series(y_test).value_counts()

-1    41676
 1     6437
Name: fit, dtype: int64

# C: large:1, else:-1

In [ ]:
# # Map 'large' to 1 and both 'small' and 'fit' to -1
# df['fit_label'] = df['fit'].map({'fit': -1, 'small': -1, 'large': 1})

In [ ]:
# One-hot encoding a categorical feature and normalizing a numerical feature
encoder = OneHotEncoder()
X_categorical = encoder.fit_transform(df[['user_id', 'item_id', 'bust size', 'rented for', 'body type', 'category']])
scaler = StandardScaler()
X_numerical = scaler.fit_transform(df[['weight', 'rating', 'size', 'age', 'height_cm']])

In [ ]:
# Combine categorical and numerical features
X_init = hstack([X_categorical, X_numerical])
X = scipy.sparse.lil_matrix(X_init)

# Prepare the target variable
# y_c = df['fit_label'].values

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, df['fit'], test_size=0.25)

In [ ]:
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Map 'fit' to 1 and both 'small' and 'large' to -1
y_train_smote = y_train_smote.replace({'fit': -1, 'small': -1, 'large': 1})
y_test = y_test.replace({'fit': -1, 'small': -1, 'large': 1})

In [ ]:
import optuna
from fastFM import sgd
from sklearn.metrics import accuracy_score
import numpy as np

def objective(trial):
    # Define the hyperparameters to be tuned
    n_iter = trial.suggest_int('n_iter', 500, 20000)
    init_stdev = trial.suggest_float('init_stdev', 0.0001, 0.25)
    l2_reg_w = trial.suggest_float('l2_reg_w', 0.0, 0.2)
    l2_reg_V = trial.suggest_float('l2_reg_V', 0.0, 20)
    rank = trial.suggest_int('rank', 1, 5)
    step_size = trial.suggest_float('step_size', 0.001, 1)

    # Initialize and train the Factorization Machine model
    fm = sgd.FMClassification(n_iter=n_iter, init_stdev=init_stdev,
                              l2_reg_w=l2_reg_w, l2_reg_V=l2_reg_V,
                              rank=rank, step_size=step_size)
    fm.fit(X_train_smote, y_train_smote)

    # Make predictions and evaluate the model
    y_pred = fm.predict(X_test)
    score = accuracy_score(y_test, np.round(y_pred))  # Assuming binary classification

    return score

# Run Optuna optimization
study = optuna.create_study(direction='maximize')  # Use 'minimize' for metrics like RMSE
study.optimize(objective, n_trials=1000)

# Best hyperparameters
best_params_3 = study.best_params
print('Best parameters:', best_params_3)


[I 2023-12-01 01:38:21,492] A new study created in memory with name: no-name-ffdf2fba-2d2d-4ac8-a393-e15b70e06604
[I 2023-12-01 01:38:21,593] Trial 0 finished with value: 0.8710951302142872 and parameters: {'n_iter': 10118, 'init_stdev': 0.20029539756250644, 'l2_reg_w': 0.0004956527184788228, 'l2_reg_V': 16.148510157050154, 'rank': 1, 'step_size': 0.03594132026612983}. Best is trial 0 with value: 0.8710951302142872.
[I 2023-12-01 01:38:21,697] Trial 1 finished with value: 0.871593955895496 and parameters: {'n_iter': 3645, 'init_stdev': 0.0625079408804949, 'l2_reg_w': 0.00033273317805180284, 'l2_reg_V': 1.9387922082820896, 'rank': 1, 'step_size': 0.7798464884171165}. Best is trial 1 with value: 0.871593955895496.
[I 2023-12-01 01:38:21,848] Trial 2 finished with value: 0.871593955895496 and parameters: {'n_iter': 4088, 'init_stdev': 0.10998848259773078, 'l2_reg_w': 0.0346139186196589, 'l2_reg_V': 7.87819519130675, 'rank': 5, 'step_size': 0.6188861248770885}. Best is trial 1 with value: 

Best parameters: {'n_iter': 17793, 'init_stdev': 0.020158426588714726, 'l2_reg_w': 0.039828653618897314, 'l2_reg_V': 2.785146332440314, 'rank': 2, 'step_size': 0.19443121576072683}


In [ ]:
print(best_params_3)

{'n_iter': 17793, 'init_stdev': 0.020158426588714726, 'l2_reg_w': 0.039828653618897314, 'l2_reg_V': 2.785146332440314, 'rank': 2, 'step_size': 0.19443121576072683}


In [ ]:
from fastFM import sgd
# Initialize and train the Factorization Machine model
fm = sgd.FMClassification(n_iter=best_params_3['n_iter'], init_stdev=best_params_3['init_stdev'], l2_reg_w=best_params_3['l2_reg_w'],
                          l2_reg_V=best_params_3['l2_reg_V'], rank=best_params_3['rank'], step_size=best_params_3['step_size'])
fm.fit(X_train_smote, y_train_smote)

# Make predictions
y_pred_c = fm.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_c)
print("Accuracy:", accuracy)

Accuracy: 0.8741920063184586


In [ ]:
pd.Series(y_pred_c).value_counts()

-1.0    47562
 1.0      551
dtype: int64

In [ ]:
pd.Series(y_test).value_counts()

-1    41935
 1     6178
Name: fit, dtype: int64

In [ ]:
y_pred_a[10:20]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.])

In [ ]:
y_pred_b[10:20]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [ ]:
y_pred_c[10:20]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [ ]:
def resolve_conflicts(y_pred_a, y_pred_b, y_pred_c):
    final_predictions = []
    for fit, small, large in zip(y_pred_a, y_pred_b, y_pred_c):
        if fit == 1 and small == -1 and large == -1:
            final_predictions.append('fit')
        elif fit == -1 and small == 1 and large == -1:
            final_predictions.append('small')
        elif fit == -1 and small == -1 and large == 1:
            final_predictions.append('large')
        else:
            final_predictions.append('fit')

    return final_predictions

# Resolve conflicts and get final predictions
final_class_predictions = resolve_conflicts(y_pred_a, y_pred_b, y_pred_c)
print(final_class_predictions)

['fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'large', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fi

In [ ]:
len(final_class_predictions)

48113

In [ ]:
pd.Series(final_class_predictions).value_counts()

fit      48082
large       24
small        7
dtype: int64

In [ ]:
y = df['fit']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, final_class_predictions)
print("Accuracy:", accuracy)

Accuracy: 0.738262008189055


In [ ]:
y_fit = ['fit'] * len(y_test)

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_fit)
print("Accuracy:", accuracy)

Accuracy: 0.7385529898364267


predcit fit/unfit by rating